# Regular expression implemented in Python for SQLite 

> The REGEXP operator is a special syntax for the regexp() user function. … the "X REGEXP Y" operator will be implemented as a call to "regexp(Y,X)".

Source: [SQL As Understood By SQLite: The LIKE, GLOB, REGEXP, and MATCH operators](http://www.sqlite.org/lang_expr.html#like)

> SQLite does not contain regular expression functionality by default.
> 
> It defines a `REGEXP` operator, but this will fail with an error message unless you or your framework [define a user function](http://www.sqlite.org/c3ref/create_function.html) called `regexp()`. How you do this will depend on your platform.

Source: [How do I use regex in a SQLite query?](https://stackoverflow.com/questions/5071601/) 

## Code adapted from these sources 

1. [sqlite3 : create function regexp with python](https://stackoverflow.com/questions/50063058/) 
2. [Problem with regexp python and sqlite](https://stackoverflow.com/questions/5365451)

In [1]:
import sqlite3
import re # Regular expression operations

First I need to define a function to implement regular expressions in SQLite. 

In [2]:
def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

I have created a small sample holding the first 100 rows of each table from `mainsurvey_sqldump.txt` ([SQLite dump, 84 MB .tar.gz file](http://xkcd.com/color/colorsurvey.tar.gz)).

In [3]:
connection = sqlite3.connect('data/sample.db')
cursor = connection.cursor()

Now I need to reference the user defined function `regexp()` to the the SQL operator `REGEXP`. 

In [4]:
connection.create_function("REGEXP", 2, regexp)

Now I can use the SQL operator `REGEXP`. 

In [5]:
cursor.execute('SELECT colorname FROM answers WHERE colorname REGEXP ?', ['[a-z]'])

In [6]:
data = cursor.fetchall()

In [7]:
print(data)

[('agua',), ('aqua',), ('aubergine',), ('baby blue',), ('black',), ('blue',), ('blue',), ('blue',), ('blue',), ('blue',), ('blue',), ('blue',), ('blue',), ('blue',), ('blue?',), ('bluish gray',), ('bright blue',), ('bright green',), ('bright green',), ('brown',), ('brown',), ('brown',), ('capslock purple',), ('carrot',), ('cornflower blue',), ('cyan',), ('dark blue',), ('dark green',), ('dark green',), ('dark iron',), ('dark orange',), ('dark tan',), ('faint violet',), ('forest green',), ('forest green',), ('fuchsia',), ('goldenrod',), ('goldish green',), ('gray',), ('green',), ('green',), ('green',), ('green',), ('green',), ('green',), ('green',), ('hot pink',), ('khaki',), ('khaki',), ('light blue',), ('light chocolate',), ('light green',), ('light purple',), ('lighter green',), ('lime green',), ('magenta',), ('magenta',), ('mauve',), ('moss green',), ('moss green',), ('mud',), ('mustard',), ('mustard yellow',), ('navy',), ('navy',), ('navy blue',), ('olive green',), ('orange',), ('p

In [8]:
len(data)

100

--- 

## Using Regular Expressions 

First I will print all entries in colorname. 

In [9]:
cursor.execute('SELECT colorname FROM names WHERE colorname REGEXP ?', ['.*'])
data = cursor.fetchall()
print(data)

[('',), ("                                                                                                      ok, now you're screwing with me",), ("                                                                               kasclaknvlzkxmnv;dfgojxx;kjvbx.vbzvlmzcnvl;dhgahxc ,,cb/lfkgjas;hgihsfblzxcb,sngas'dgpsidjga;dflkasfgjafg'afdga;dfgadfglkdjfg;dlfkjgafdg;lkadfgja'fg;akdfjgadfl'gdfakgjdfl'gkjdfg/m,nbv.vbmnfb'dfgkldfhdgf';fg'adfg;kjf';lgfkgs",), ('                                 (o")',), ('                                ("o)',), ('                               ("o)',), ('                               (o")',), ('                               >>#',), ('                         >>==n;)',), ('                        (o")',), ('                      ("o)',), ('                     >>===="o)',), ('                   (o")',), ('                  >>====>("o)',), ('                (o")',), ('               >>====>   ("o)',), ('               yellow',), ('             ("o)',), ('    

Next I will only print those entries in `colorname` that include **at least one** lowercase letter from a to z. 

This excludes: 
- `('',)`
- `('                               >>#',)`
- `(' !!!!!!1!!]',)`
- `(' +',)`
- `(' ,',)`
- `(' 486',)`

In [10]:
cursor.execute('SELECT colorname FROM names WHERE colorname REGEXP ?', ['[a-z]'])
data = cursor.fetchall()
print(data)

[("                                                                                                      ok, now you're screwing with me",), ("                                                                               kasclaknvlzkxmnv;dfgojxx;kjvbx.vbzvlmzcnvl;dhgahxc ,,cb/lfkgjas;hgihsfblzxcb,sngas'dgpsidjga;dflkasfgjafg'afdga;dfgadfglkdjfg;dlfkjgafdg;lkadfgja'fg;akdfjgadfl'gdfakgjdfl'gkjdfg/m,nbv.vbmnfb'dfgkldfhdgf';fg'adfg;kjf';lgfkgs",), ('                                 (o")',), ('                                ("o)',), ('                               ("o)',), ('                               (o")',), ('                         >>==n;)',), ('                        (o")',), ('                      ("o)',), ('                     >>===="o)',), ('                   (o")',), ('                  >>====>("o)',), ('                (o")',), ('               >>====>   ("o)',), ('               yellow',), ('             ("o)',), ('          (o")',), ('         (o")',), ('       ("o)

In [11]:
len(data)

94

For better legibility I will remove padding whitespaces. Beware that the regular expression is matched *after* the trimming! 

`trim()` removes whitespaces *before* and *after* (i.e. padding) the string matching the regular expression. See https://www.techonthenet.com/sqlite/functions/trim.php

In [12]:
cursor.execute('SELECT trim(colorname) AS trimmed_cname FROM names WHERE trimmed_cname REGEXP ?', ['[a-z]'])
data = cursor.fetchall()
print(data)

[("ok, now you're screwing with me",), ("kasclaknvlzkxmnv;dfgojxx;kjvbx.vbzvlmzcnvl;dhgahxc ,,cb/lfkgjas;hgihsfblzxcb,sngas'dgpsidjga;dflkasfgjafg'afdga;dfgadfglkdjfg;dlfkjgafdg;lkadfgja'fg;akdfjgadfl'gdfakgjdfl'gkjdfg/m,nbv.vbmnfb'dfgkldfhdgf';fg'adfg;kjf';lgfkgs",), ('(o")',), ('("o)',), ('("o)',), ('(o")',), ('>>==n;)',), ('(o")',), ('("o)',), ('>>===="o)',), ('(o")',), ('>>====>("o)',), ('(o")',), ('>>====>   ("o)',), ('yellow',), ('("o)',), ('(o")',), ('(o")',), ('("o)',), ('>>====>            ("o)',), ('(o")',), ('what would you call this color?',), ('1. turquoise',), ('bmb',), ('cornflowerblue',), ('h',), ('n mn',), ('orange',), ('turquoise',), ('#100c08',), ('#38acec',), ('#b57281',), ('#dd00ff (yes i looked it up)',), ('#e4d96f',), ('("o)',), ('(o")',), ('... some off green',), ('5gt',), ('68f79oibjl',), ('83 2inf',), ('a',), ('a cross between terra cotta and light pink',), ('a little bit light blue',), ('a mix of blue and purple',), ('a;sodifjoawiemv',), ('akjd f;oiaeg',), ('

In [13]:
len(data)

94

`[a-z]` includes all lower case letter from a to z. 
`.*` combines `.` for `Any character except new line (\n)` and `*` for `0 or more`. 

Next I will only print those entries in `colorname` 
- that **start and end with a lowercase letter** from a to z 
- that consist of **at least two letters** 

This excludes: 
- `('(o")',)`
- `('("o)',)`
- `('("o)',)`
- `('(o")',)`
- `('>>==n;)',)`
- `('(o")',)`
- `('("o)',)`
- `('>>===="o)',)`
- `('(o")',)`
- `('>>====>("o)',)`
- `('(o")',)`
- `('>>====>   ("o)',)`
- `('("o)',)`
- `('(o")',)`
- `('(o")',)`
- `('("o)',)`
- `('>>====>            ("o)',)`
- `('(o")',)`
- `('what would you call this color?',)`
- `('1. turquoise',)`
- `('h',)`
- `('#100c08',)`
- `('#38acec',)`
- `('#b57281',)`
- `('#dd00ff (yes i looked it up)',)`
- `('#e4d96f',)`
- `('("o)',)`
- `('(o")',)`
- `('... some off green',)`
- `('5gt',)`
- `('68f79oibjl',)`
- `('83 2inf',)`
- `('a',)`
- `('b',)`
- `('blue!',)`
- `('blue.',)`

In [14]:
cursor.execute('SELECT trim(colorname) AS trimmed_cname FROM names WHERE trimmed_cname REGEXP ?', ['^[a-z].*[a-z]$'])
data = cursor.fetchall()
print(data)

[("ok, now you're screwing with me",), ("kasclaknvlzkxmnv;dfgojxx;kjvbx.vbzvlmzcnvl;dhgahxc ,,cb/lfkgjas;hgihsfblzxcb,sngas'dgpsidjga;dflkasfgjafg'afdga;dfgadfglkdjfg;dlfkjgafdg;lkadfgja'fg;akdfjgadfl'gdfakgjdfl'gkjdfg/m,nbv.vbmnfb'dfgkldfhdgf';fg'adfg;kjf';lgfkgs",), ('yellow',), ('bmb',), ('cornflowerblue',), ('n mn',), ('orange',), ('turquoise',), ('a cross between terra cotta and light pink',), ('a little bit light blue',), ('a mix of blue and purple',), ('a;sodifjoawiemv',), ('akjd f;oiaeg',), ('anastasia lavender',), ('aqua',), ('ash',), ('baby blue',), ('baby vomit green',), ('bark',), ('battleship grey',), ('behbeh bloo',), ('beige',), ('beige brown',), ('beigh peach',), ('between sky and marine blue',), ('bilious green',), ('black',), ('blood red',), ('bloom',), ('blue',), ('blue -lightish',), ('blue green',), ('blue green blue',), ('blue green green',), ('blue purple',), ('blue-gray',), ('blue-green',), ('blue-turquoise',), ('bluebatone',), ('bluish',), ('blurple',), ('bnjk65

In [15]:
len(data)

58

`[:alpha:]` is a [POSIX](https://en.wikipedia.org/wiki/POSIX) standard for `All letters`, and supposedly identical to `[a-zA-Z]` <sup>[POSIX Bracket Expressions](https://www.regular-expressions.info/posixbrackets.html)</sup>

I found however that the results diverge. 

In [22]:
cursor.execute('SELECT trim(colorname) AS trimmed_cname FROM names WHERE trimmed_cname REGEXP ?', ['.*'])
data = cursor.fetchall()
print(data)

[('',), ("ok, now you're screwing with me",), ("kasclaknvlzkxmnv;dfgojxx;kjvbx.vbzvlmzcnvl;dhgahxc ,,cb/lfkgjas;hgihsfblzxcb,sngas'dgpsidjga;dflkasfgjafg'afdga;dfgadfglkdjfg;dlfkjgafdg;lkadfgja'fg;akdfjgadfl'gdfakgjdfl'gkjdfg/m,nbv.vbmnfb'dfgkldfhdgf';fg'adfg;kjf';lgfkgs",), ('(o")',), ('("o)',), ('("o)',), ('(o")',), ('>>#',), ('>>==n;)',), ('(o")',), ('("o)',), ('>>===="o)',), ('(o")',), ('>>====>("o)',), ('(o")',), ('>>====>   ("o)',), ('yellow',), ('("o)',), ('(o")',), ('(o")',), ('("o)',), ('>>====>            ("o)',), ('(o")',), ('what would you call this color?',), ('1. turquoise',), ('bmb',), ('cornflowerblue',), ('h',), ('n mn',), ('orange',), ('turquoise',), ('!!!!!!1!!]',), ('#100c08',), ('#38acec',), ('#b57281',), ('#dd00ff (yes i looked it up)',), ('#e4d96f',), ('("o)',), ('(o")',), ('+',), (',',), ('... some off green',), ('486',), ('5gt',), ('68f79oibjl',), ('83 2inf',), ('a',), ('a cross between terra cotta and light pink',), ('a little bit light blue',), ('a mix of blu

In [23]:
len(data)

100

In [18]:
cursor.execute('SELECT trim(colorname) AS trimmed_cname FROM names WHERE trimmed_cname REGEXP ?', ['[a-zA-Z]'])
data = cursor.fetchall()
print(data)

[("ok, now you're screwing with me",), ("kasclaknvlzkxmnv;dfgojxx;kjvbx.vbzvlmzcnvl;dhgahxc ,,cb/lfkgjas;hgihsfblzxcb,sngas'dgpsidjga;dflkasfgjafg'afdga;dfgadfglkdjfg;dlfkjgafdg;lkadfgja'fg;akdfjgadfl'gdfakgjdfl'gkjdfg/m,nbv.vbmnfb'dfgkldfhdgf';fg'adfg;kjf';lgfkgs",), ('(o")',), ('("o)',), ('("o)',), ('(o")',), ('>>==n;)',), ('(o")',), ('("o)',), ('>>===="o)',), ('(o")',), ('>>====>("o)',), ('(o")',), ('>>====>   ("o)',), ('yellow',), ('("o)',), ('(o")',), ('(o")',), ('("o)',), ('>>====>            ("o)',), ('(o")',), ('what would you call this color?',), ('1. turquoise',), ('bmb',), ('cornflowerblue',), ('h',), ('n mn',), ('orange',), ('turquoise',), ('#100c08',), ('#38acec',), ('#b57281',), ('#dd00ff (yes i looked it up)',), ('#e4d96f',), ('("o)',), ('(o")',), ('... some off green',), ('5gt',), ('68f79oibjl',), ('83 2inf',), ('a',), ('a cross between terra cotta and light pink',), ('a little bit light blue',), ('a mix of blue and purple',), ('a;sodifjoawiemv',), ('akjd f;oiaeg',), ('

In [19]:
len(data)

94

In [20]:
cursor.execute('SELECT trim(colorname) AS trimmed_cname FROM names WHERE trimmed_cname REGEXP ?', ['[:alpha:]'])
data = cursor.fetchall()
print(data)

[("ok, now you're screwing with me",), ("kasclaknvlzkxmnv;dfgojxx;kjvbx.vbzvlmzcnvl;dhgahxc ,,cb/lfkgjas;hgihsfblzxcb,sngas'dgpsidjga;dflkasfgjafg'afdga;dfgadfglkdjfg;dlfkjgafdg;lkadfgja'fg;akdfjgadfl'gdfakgjdfl'gkjdfg/m,nbv.vbmnfb'dfgkldfhdgf';fg'adfg;kjf';lgfkgs",), ('yellow',), ('what would you call this color?',), ('cornflowerblue',), ('h',), ('orange',), ('#38acec',), ('#dd00ff (yes i looked it up)',), ('68f79oibjl',), ('a',), ('a cross between terra cotta and light pink',), ('a little bit light blue',), ('a mix of blue and purple',), ('a;sodifjoawiemv',), ('akjd f;oiaeg',), ('anastasia lavender',), ('aqua',), ('ash',), ('baby blue',), ('baby vomit green',), ('bark',), ('battleship grey',), ('behbeh bloo',), ('beigh peach',), ('between sky and marine blue',), ('bilious green',), ('black',), ('blood red',), ('bloom',), ('blue',), ('blue -lightish',), ('blue green',), ('blue green blue',), ('blue green green',), ('blue purple',), ('blue!',), ('blue-gray',), ('blue-green',), ('blue-t

In [21]:
len(data)

57

The results puzzled me until I realized that `[:alpha:]` is not recognized as a POSIX, so all strings are excluded that do not contain `a`, `l`, `p` or `h`. 
- e.g. `('#100c08',)` but not `('#38acec',)`
- e.g. `('... some off green',)` but not `('a;sodifjoawiemv',)`

If in doubt check e.g. with [Rubular](http://rubular.com/). 